#

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import math
from functools import partial
from typing import Iterable, Callable, Optional
from numbers import Number

In [2]:
class PredictionTask:
    def __init__(self, input, outcome=None):
        self.A = input
        self.B = outcome


class WeightedPredictionClass:
    def __init__(self, tasks: Iterable[PredictionTask], weights: Optional[Iterable[Number]] = None):
        self.tasks = tasks
        for task in self.tasks:
            if task.B is None:
                raise ValueError("All WeightedPredictionClass task outcomes must be known.")
        J = len(tasks)
        if weights is not None:
            if len(weights) != J:
                raise ValueError(f"len(weights), {len(weights)}, must equal len(tasks), {J}")
            self.weights = np.array(weights) / sum(weights)
        else:
            self.weights = np.ones(J)/J

    def predictive_success(self, predicted_log_likelihood: Callable[[PredictionTask], float]) -> float:
        return sum(w * predicted_log_likelihood(task) 
                   for w, task in zip(self.weights, self.tasks)
                   if task.B is not None)

In [3]:
def design_matrix(model_name, x):
    if model_name == "POLY-4":
        X = np.column_stack([np.ones(len(x)), x, x**2, x**3, x**4])
    elif model_name == "CUBIC":
        X = np.column_stack([np.ones(len(x)), x, x**2, x**3])
    elif model_name == "PAR":
        X = np.column_stack([np.ones(len(x)), x, x**2])
    elif model_name == "LIN":
        X = np.column_stack([np.ones(len(x)), x])
    elif model_name == "LIN0":
        X = x.reshape(-1, 1)  # Only x as a feature (no intercept)
    else:
        raise NotImplementedError(model_name)
    return X

def log_likelihood(y_pred, y, sigma2) -> float:
    """Average log-likelihood"""
    pi = math.pi
    ln = math.log
    MSE = np.mean((y_pred - y)**2)
    return -(1/2)*(ln(2*pi*sigma2) + MSE/sigma2)


def predicted_ll(model_name, task: PredictionTask) -> float:
    x = task.A['x_known']
    y = task.A['y_known']
    x_out = task.A['x_to_predict']
    fit_results = sm.OLS(y, design_matrix(model_name, x)).fit()
    y_pred = fit_results.predict(design_matrix(model_name, x_out))
    return log_likelihood(y_pred, task.B, fit_results.mse_resid)

In [4]:
def generate_y(x, noise_std=0.0):
    """Generates data based on the specified model."""
    y = 0.5 + 0.5 * np.tanh(x - 2)
    if noise_std:
        y += np.random.normal(0, noise_std, size=len(x))
    return y

def generate_x(x_range, step=0.1):
    return np.round(np.arange(x_range[0], x_range[1] + step, step), decimals=1)

In [5]:
# Define x ranges
x_ranges = {
    "X0": (0, 3.5),
    "Xtarg": (3.6, 5),
    "Xall": (0, 5),
    "Xcal": (0, 2.5),
    "Xgen": (2.6, 3.5),
    "Xcal1": (0, 2.3),
    "Xgen1": (2.4, 3.5),
    "Xcal2": (0, 2.4),
    "Xgen2": (2.5, 3.5),
    "Xcal3": (0, 2.6),
    "Xgen3": (2.7, 3.5),
}

x_all = generate_x(x_ranges["Xall"])
y_true = generate_y(x_all)
f_dict = {x: y for x, y in zip(x_all, y_true)}
#plt.figure(figsize=(12, 8))
#plt.plot(x_all, y_true, label="TRUE", color="blue", linewidth=2)

def task(in_range_name: str, out_range_name: str) -> PredictionTask:
    x_known = generate_x(x_ranges[in_range_name])
    x_to_predict = generate_x(x_ranges[out_range_name])
    return PredictionTask(
        dict(x_known=x_known,
             y_known=np.array([f_dict[x_i] for x_i in x_known]),
             x_to_predict=x_to_predict
        ),
        np.array([f_dict[x_i] for x_i in x_to_predict])
    )

In [6]:
predicted_ll("POLY-4", task("Xcal", "Xgen"))

np.float64(-2455.6705643687956)

In [ ]:
task_of_interest = task("X0", "Xall")
plt.scatter(task_of_interest.A['x_known'], task_of_interest.A['y_known'], label="TRUE", color="blue")
plt.xlim(0, 5)
plt.ylim(0, 1)
plt.show()

In [8]:
def run_gen_simulation(range_names, models):
    """Runs the simulation and returns the results."""

    results_dict = {}
    for model_name in models:
        results_dict[model_name] = {}
        names = []
        for item in range_names: #range_name, x_range in _x_ranges.items():
            name = f"{item[0]}->{item[1]}"
            names.append(name)
            results_dict[model_name][name] = {
                "ll_score": predicted_ll(model_name, task(item[0], item[1]))
                #"x": x,  # Store x and y for plotting
                #"y": y,
            }
    display(pd.DataFrame([[model] + [results_dict[model][range_name]["ll_score"] for range_name in names] for model in models]))
    return results_dict

sim_models = list(reversed(["POLY-4", "CUBIC", "PAR", "LIN", "LIN0"]))
sim_range_names = [("Xcal", "Xgen"), ("X0", "Xtarg"), ("X0", "Xall")]
results_gen = run_gen_simulation(sim_range_names, sim_models)

In [9]:
temp0 = np.exp(np.array([[results_gen[model][range_name]["ll_score"] for range_name in results_gen[model]] for model in sim_models]))
temp0

array([[6.70479807e-001, 4.62091813e-001, 1.59990715e+000],
       [2.43587705e+000, 1.56743339e-004, 1.97554521e-001],
       [5.15419552e-206, 2.09065481e-019, 9.49878922e-006],
       [5.62581763e-152, 5.31134108e-111, 2.62173083e-032],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000]])

In [10]:
temp0[:, 0]/temp0[1, 0]

array([2.75251908e-001, 1.00000000e+000, 2.11595061e-206, 2.30956551e-152,
       0.00000000e+000])

In [11]:
step = 0.1
explore_range_names = []
for i in range(0, 10):
    cal_end = round(2.5 + step * i, 1)
    gen_start = round(cal_end + step, 1)
    cal_name = f"Xcal{cal_end:.1f}"
    gen_name = f"Xgen{cal_end:.1f}"
    x_ranges[cal_name] = (0, cal_end)
    x_ranges[gen_name] = (gen_start, 3.5)
    explore_range_names.append((cal_name, gen_name))
results_gen2 = run_gen_simulation(explore_range_names, sim_models)

In [12]:
step = 0.1
tasks = []
for i in range(0, 10):
    cal_end = round(2.5 + step * i, 1)
    gen_start = round(cal_end + step, 1)
    cal_name = f"Xcal{cal_end:.1f}"
    gen_name = f"Xgen{cal_end:.1f}"
    x_ranges[cal_name] = (0, cal_end)
    x_ranges[gen_name] = (gen_start, 3.5)
    tasks.append(task(cal_name, gen_name))
pclass = WeightedPredictionClass(tasks)
[math.exp(pclass.predictive_success(partial(predicted_ll, model_name))) for model_name in sim_models]

[2.3789457135907206,
 4.2286272743981925,
 5.199645649009759e-48,
 1.3160873860840432e-29,
 0.0]

In [13]:
temp = np.exp(np.mean(np.array([[results_gen2[model][range_name]["ll_score"] for range_name in results_gen2[model]] for model in sim_models]), axis=1))
temp

array([2.37894571e+00, 4.22862727e+00, 5.19964565e-48, 1.31608739e-29,
       0.00000000e+00])

In [14]:
temp/temp[1]

array([5.62581083e-01, 1.00000000e+00, 1.22962969e-48, 3.11232771e-30,
       0.00000000e+00])

In [15]:
math.log(10)

2.302585092994046

In [16]:
math.log(2)

0.6931471805599453

In [17]:
[pclass.predictive_success(partial(predicted_ll, model_name)) for model_name in sim_models]

[np.float64(0.866657412056983),
 np.float64(1.441877418970849),
 np.float64(-108.87549398486995),
 np.float64(-66.50030446332934),
 np.float64(-1931.8996035482776)]

In [18]:
x_ranges

{'X0': (0, 3.5),
 'Xtarg': (3.6, 5),
 'Xall': (0, 5),
 'Xcal': (0, 2.5),
 'Xgen': (2.6, 3.5),
 'Xcal1': (0, 2.3),
 'Xgen1': (2.4, 3.5),
 'Xcal2': (0, 2.4),
 'Xgen2': (2.5, 3.5),
 'Xcal3': (0, 2.6),
 'Xgen3': (2.7, 3.5),
 'Xcal2.5': (0, 2.5),
 'Xgen2.5': (2.6, 3.5),
 'Xcal2.6': (0, 2.6),
 'Xgen2.6': (2.7, 3.5),
 'Xcal2.7': (0, 2.7),
 'Xgen2.7': (2.8, 3.5),
 'Xcal2.8': (0, 2.8),
 'Xgen2.8': (2.9, 3.5),
 'Xcal2.9': (0, 2.9),
 'Xgen2.9': (3.0, 3.5),
 'Xcal3.0': (0, 3.0),
 'Xgen3.0': (3.1, 3.5),
 'Xcal3.1': (0, 3.1),
 'Xgen3.1': (3.2, 3.5),
 'Xcal3.2': (0, 3.2),
 'Xgen3.2': (3.3, 3.5),
 'Xcal3.3': (0, 3.3),
 'Xgen3.3': (3.4, 3.5),
 'Xcal3.4': (0, 3.4),
 'Xgen3.4': (3.5, 3.5)}

In [19]:
step = 0.1
explore_range_names = []
for i in range(0, 10):
    cal_end = round(2.5 + step * i, 1)
    gen_start = cal_end #round(cal_end + step, 1)
    cal_name = f"Xcal{cal_end:.1f}orig"
    gen_name = f"Xgen{cal_end:.1f}orig"
    x_ranges[cal_name] = (0, cal_end)
    x_ranges[gen_name] = (gen_start, 3.5)
    explore_range_names.append((cal_name, gen_name))
results_gen3 = run_gen_simulation(explore_range_names, sim_models)